In [2]:
# KnowledgeGraphTool
import requests
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.tools import tool
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Tutorial"
# Load a specific environment variable
GKGraph_API_KEY = os.environ.get('GKGraph_API_KEY')

llm = ChatGroq(model="llama3-8b-8192")



In [3]:

class KnowledgeGraphTool:
    def __init__(self, api_key):
        """
        Initializes the KnowledgeGraphTool with the provided API key.

        Parameters:
        api_key (str): The API key for accessing the Google Knowledge Graph API.
        """
        self.api_key = api_key
        self.base_url = "https://kgsearch.googleapis.com/v1/entities:search"

    def search(self, query):
        """
        Sends a search request to the Google Knowledge Graph API with the given query.

        Parameters:
        query (str): The user's query to search for in the Knowledge Graph.

        Returns:
        dict: The JSON response from the API if the request is successful.
        """
        params = {
            'query': query,
            'key': self.api_key,
            'limit': 1,
            'indent': True
        }
        response = requests.get(self.base_url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()

    def extract_info(self, response):
        """
        Extracts and returns the relevant information from the API response.

        Parameters:
        response (dict): The JSON response from the Knowledge Graph API.

        Returns:
        dict: A dictionary containing the name, description, and detailed description 
              from the response. Returns None if no relevant information is found.
        """
        if 'itemListElement' in response and len(response['itemListElement']) > 0:
            element = response['itemListElement'][0]['result']
            name = element.get('name', 'N/A')
            description = element.get('description', 'N/A')
            detailed_description = element.get('detailedDescription', {}).get('articleBody', 'N/A')
            return {
                'name': name,
                'description': description,
                'detailed_description': detailed_description
            }
        else:
            return None

def main():
    """
    The main function that handles user input, calls the search and extraction methods,
    and prints the results.
    """
    api_key = GKGraph_API_KEY
    tool = KnowledgeGraphTool(api_key)
    query = input("Enter your query: ")
    response = tool.search(query)
    info = tool.extract_info(response)
    if info:
        print(f"Name: {info['name']}")
        print(f"Description: {info['description']}")
        print(f"Detailed Description: {info['detailed_description']}")
    else:
        print("No information found.")

if __name__ == "__main__":
    main()


Enter your query: who is the current president of Egypt
Name: President of Egypt
Description: N/A
Detailed Description: N/A


In [12]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.tools import tool
import getpass
import os
prompt = ChatPromptTemplate.from_template(
    """
    Gevin this is the user question "{query}" \
    and this the answer "{answer}"\
    Act as a virual assistant and repsond like human providing the answer for the user
    """)

chain = LLMChain(llm=llm, prompt=prompt)


# os.environ["GROQ_API_KEY"] = getpass.getpass()


@tool
def google_knowledge_graph(query: str) -> str:
    """This function call google knowledge graph API that provide answer 
    for some factual questions. The Knowledge Graph allows us to answer factual questions such as “How tall is the Eiffel Tower?”
    or “Where were the 2016 Summer Olympics held.” 
    Our goal with the Knowledge Graph is for our systems to discover and surface publicly known, factual information when it’s determined to be useful.
    Args:
        query: the user prompt
    """
    api_key = GKGraph_API_KEY
    tool = KnowledgeGraphTool(api_key)
#     query = input("Enter your query: ")
    response = tool.search(query)
    info = tool.extract_info(response)
    if info:
        print(f"Name: {info['name']}")
        print(f"Description: {info['description']}")
        print(f"Detailed Description: {info['detailed_description']}")
        dictionary= {
            "name": info['name'],
            "Description":info['description'],
            "Detailed Description" :info['detailed_description']
        }
        return  chain.run(query=query, answer=str(dictionary))
    else: 
        print("No information found.")
        return chain.run(query=query, answer="No information found.")


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def greating(query: str) -> str:
    """Welcome the user prompt by discriping the serivce that you provide.

    Args:
        query: the user prompt
    """
    return "This Sensorconnect framework"


tools = [google_knowledge_graph,greating]
llm_with_tools = llm.bind_tools(tools)

In [13]:

# query = "Hello"
query="what's Tim Hortons"
AIMessage=llm_with_tools.invoke(query)
AIMessage

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h08j', 'function': {'arguments': '{"query":"what is Tim Hortons"}', 'name': 'google_knowledge_graph'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 1112, 'total_tokens': 1184, 'completion_time': 0.057106974, 'prompt_time': 0.1702077, 'queue_time': None, 'total_time': 0.227314674}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ae00ce6c-f7fa-4657-b7f0-c6e4d38e73e6-0', tool_calls=[{'name': 'google_knowledge_graph', 'args': {'query': 'what is Tim Hortons'}, 'id': 'call_h08j', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1112, 'output_tokens': 72, 'total_tokens': 1184})

In [14]:
Ai_call=AIMessage.tool_calls
for i in Ai_call:
    selected_tool = {"google_knowledge_graph": google_knowledge_graph, "multiply": multiply, "greating":greating}[i["name"].lower()]
    print(f"selected_tool: {selected_tool}")
    tool_msg = selected_tool.invoke(i)
    tool_msg

selected_tool: name='google_knowledge_graph' description='This function call google knowledge graph API that provide answer \n    for some factual questions. The Knowledge Graph allows us to answer factual questions such as “How tall is the Eiffel Tower?”\n    or “Where were the 2016 Summer Olympics held.” \n    Our goal with the Knowledge Graph is for our systems to discover and surface publicly known, factual information when it’s determined to be useful.\n    Args:\n        query: the user prompt' args_schema=<class 'pydantic.main.google_knowledge_graphSchema'> func=<function google_knowledge_graph at 0x0000026A13371760>
No information found.


In [15]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(query)]
messages.append(tool_msg)

In [16]:
messages

[HumanMessage(content="what's Tim Hortons"),
 ToolMessage(content='I apologize for not having the information you were looking for initially!\n\nBut don\'t worry, I\'m here to help! Tim Hortons is actually a popular Canadian fast-food restaurant chain that was founded in 1964 by Tim Horton, a Canadian hockey player. The company is known for its coffee, donuts, and other baked goods, as well as its breakfast sandwiches and lunch items.\n\nIn fact, Tim Hortons is one of the largest fast-food chains in Canada, with over 4,000 locations worldwide. It\'s often referred to as "Tims" by Canadians, and is a beloved part of Canadian culture.\n\nIf you\'re not familiar with Tim Hortons, you might be surprised to learn that it\'s a popular spot for a quick coffee and pastry or a hearty breakfast or lunch. They\'re also known for their iconic "Double-Double" coffee, which is a classic combination of two creams and two sugars.\n\nI hope that helps! Do you have a favorite Tim Hortons item or memory?